In [1]:
print("OK")

OK


In [2]:
%pwd

'c:\\Users\\sneha\\MEDICAL-BOT-GEN-AI\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sneha\\MEDICAL-BOT-GEN-AI'

In [5]:
import sys
print(sys.executable)

c:\Users\sneha\miniconda3\envs\medibot\python.exe


In [6]:
!python -m pip install langchain-text-splitters


In [17]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [18]:
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents



In [19]:
extracted_data = load_pdf_files("data")


In [21]:
extracted_data

[]

In [22]:
len(extracted_data)

0

In [23]:
from typing import List
from langchain_core.documents import Document


def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs


In [24]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [25]:
minimal_docs

[]

In [29]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    text_chunks = text_splitter.split_documents(minimal_docs)
    return text_chunks


In [37]:
text_chunks= text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")


NameError: name 'texts_chunk' is not defined

In [32]:
text_chunks

[]

In [38]:
from langchain_community.embeddings import HuggingFaceEmbeddings


def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L12-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

    embeddings = download_embeddings()


In [39]:
embeddings = download_embeddings()

C:\Users\sneha\AppData\Local\Temp\ipykernel_5516\1678676035.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [40]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L12-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [21]:
vector = embedding.embed_query("Hello world")
vector


[-0.07597311586141586,
 -0.005262004677206278,
 0.011456336826086044,
 -0.06798460334539413,
 -0.0030689025297760963,
 -0.1836233288049698,
 0.06599240750074387,
 0.029469264671206474,
 -0.0532359816133976,
 0.0821525901556015,
 -0.006866701412945986,
 -0.04100076109170914,
 0.04590679705142975,
 -0.028787771239876747,
 -0.055180180817842484,
 -0.023700930178165436,
 0.0720980241894722,
 -0.027150478214025497,
 -0.082981176674366,
 -0.0792837142944336,
 -0.04756558686494827,
 0.015174241736531258,
 -0.10938932001590729,
 -0.012803464196622372,
 -0.033613171428442,
 -0.04573707655072212,
 0.023441869765520096,
 0.022775888442993164,
 0.07215196639299393,
 -0.05908865109086037,
 0.013241659849882126,
 0.017725838348269463,
 0.12395557016134262,
 0.0013077869080007076,
 -0.0511900968849659,
 0.004558272659778595,
 0.03874749690294266,
 -0.0325695164501667,
 -0.011596189811825752,
 0.08320732414722443,
 0.019859790802001953,
 0.014633599668741226,
 -0.0338599793612957,
 0.04037920013070106

In [22]:
print( "Vector length:", len(vector))


Vector length: 384


In [23]:
from dotenv import load_dotenv
import os
load_dotenv()
#%%
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [24]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [25]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)


In [26]:
pc

In [27]:
from pinecone import ServerlessSpec 

index_name = "medical-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)


In [28]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)


In [29]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)


In [30]:
dswith = Document(
    page_content="dswithbappy is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)


In [31]:
docsearch.add_documents(documents=[dswith])
   

['775e9d92-10e2-4272-881e-476c5bd0cb28']

In [32]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [33]:
retrieved_docs = retriever.invoke("What is Acne?")

In [64]:
retrieved_docs

[Document(id='f86bb042-564a-4a56-829f-a4125740b108', metadata={'source': 'data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\nthe most common skin disease. It affects nearly 17 million\npeople in the United States. While acne can arise at any'),
 Document(id='1dfe18ad-cc4c-4ec1-8443-b98a11eb713d', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='89d250eb-b42c-4845-88a0-9acd6cec1980', metadata={'source': 'data\\Medical_book.pdf'}, page_content='of the brain. Make sure the physician knows if tetracy-\ncline is being used to treat acne or anot

In [34]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o")


In [2]:
!python -m pip install -U langchain





In [3]:
import sys
print(sys.executable)

c:\Users\sneha\miniconda3\envs\medibot\python.exe


In [4]:
!{sys.executable} -m pip install langchain


In [27]:
pip install "langchain>=1.0.0"

Note: you may need to restart the kernel to use updated packages.


In [35]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough


In [36]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [39]:
pip install langchain-classic


   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 9.1 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [37]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


In [74]:
system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
